## data for uncolored top partner

In [1]:
versioninfo()

Julia Version 1.0.3
Commit 099e826241 (2018-12-18 01:34 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-6.0.0 (ORCJIT, skylake)


In [2]:
using Distributed
n_add = 4 - nprocs()
addprocs(n_add)
@show nprocs()
;

nprocs() = 4


In [3]:
@everywhere include("so5.jl");

In [4]:
@everywhere begin

function full_V(p::Float64, t::Float64, args::Params)
    return -2*3*V(p, t, args, top_s, top_denom) +
           -2*V(p, t, args, partner_s, partner_denom) +
            3*V(p, t, args, boson_s, W_denom) +
            3/2*V(p, t, args, boson_s, Z_denom)
end

function full_dV(p::Float64, t::Float64, args::Params)
    return -2*3*dV(p, t, args, top_angles, top_denom) +
           -2*dV(p, t, args, partner_angles, partner_denom) +
            3*dV(p, t, args, boson_angles, W_denom) +
            3/2*dV(p, t, args, boson_angles, Z_denom)
end

function full_ddV(p::Float64, t::Float64, args::Params)
    return -2*3*ddV(p, t, args, top_angles, top_denom) +
           -2*ddV(p, t, args, partner_angles, partner_denom) +
            3*ddV(p, t, args, boson_angles, W_denom) +
            3/2*ddV(p, t, args, boson_angles, Z_denom)
end

function full_dddV(p::Float64, t::Float64, args::Params)
    return -2*3*dddV(p, t, args, top_angles, top_denom) +
           -2*dddV(p, t, args, partner_angles, partner_denom) +
            3*dddV(p, t, args, boson_angles, W_denom) +
            3/2*dddV(p, t, args, boson_angles, Z_denom)
end
    
end
;

In [5]:
# mutable struct Params
#     zR    ::Float64
#     ctop  ::Float64
#     cvar  ::Float64
#     aL    ::Float64
#     aB    ::Float64
#     af    ::Float64
#     minphi::Float64
#     mh    ::Float64
#     f     ::Float64
#     valid ::Bool
# end

In [6]:
@everywhere begin
    using SharedArrays
    ctop_list = collect(0.39:0.0025:0.71)
    af_list = collect(range(20.,stop=0., length=41)) # Note, af is in the descending order
    c_len = length(ctop_list)
    af_len = length(af_list)
    ctop=1; cT=2; aL=3; aB=4; af=5; minphi=6; higgsmass=7; zR=8;    
end
data = SharedArray{Float64,3}((8, c_len, af_len))
;

In [7]:
@everywhere function inner_loop(i::Int64, j::Int64, data::SharedArray{Float64,3})
    arg1 = Params(0.,0.,0.,0.,0.,0.,0.,0.,0.,false)
    arg1.af = af_list[j]
    arg1.ctop = ctop_list[i]
    arg1.aL = findaL(arg1)
    arg1.aB = arg1.aL
    solve(arg1)
    @printf "ctop = %.4f : " ctop_list[i]
    print(printsolve(arg1))
    
    if arg1.valid
        data[ctop,i,j]      = arg1.ctop
        data[cT,i,j]        = arg1.cvar
        data[aL,i,j]        = arg1.aL
        data[aB,i,j]        = arg1.aB
        data[af,i,j]        = arg1.af
        data[minphi,i,j]    = arg1.minphi
        data[higgsmass,i,j] = arg1.mh
        data[zR,i,j]        = arg1.zR 
    end
end
;

In [8]:
scan_width = 15

j=1
println(af_list[j])
pmap((args)->inner_loop(args...), [(i, j, data) for i in 1:scan_width])

j=41
println(af_list[j])
pmap((args)->inner_loop(args...), [(i, j, data) for i in c_len-scan_width:c_len])
;

20.0
      From worker 3:	ctop = 0.3925 : cT = 0.53558, minphi = 0.05155, zR = 0.20597, mh = 0.12506, valid 
      From worker 4:	ctop = 0.3950 : cT = 0.54037, minphi = 0.05570, zR = 0.22066, mh = 0.12510, valid 
      From worker 2:	ctop = 0.3900 : cT = 0.53078, minphi = 0.04753, zR = 0.19152, mh = 0.12510, valid 
      From worker 3:	ctop = 0.3975 : cT = 0.54520, minphi = 0.06062, zR = 0.23809, mh = 0.12509, valid 
      From worker 4:	ctop = 0.4000 : cT = 0.55002, minphi = 0.06575, zR = 0.25596, mh = 0.12508, valid 
      From worker 2:	ctop = 0.4025 : cT = 0.55487, minphi = 0.07145, zR = 0.27572, mh = 0.12506, valid 
      From worker 4:	ctop = 0.4075 : cT = 0.56470, minphi = 0.08478, zR = 0.32128, mh = 0.12509, valid 
      From worker 2:	ctop = 0.4100 : cT = 0.56970, minphi = 0.09249, zR = 0.34724, mh = 0.12508, valid 
      From worker 3:	ctop = 0.4050 : cT = 0.55975, minphi = 0.07774, zR = 0.29732, mh = 0.12510, valid 
      From worker 4:	ctop = 0.4125 : cT = 0.57477, minphi =

In [9]:
@everywhere function set_cstart_idx(j::Int64, data::SharedArray{Float64,3})
    if j < 2
        return 1
    end
    return findfirst(x -> x > 0.23, data[zR, :, j-1])
end

@everywhere cspan(j::Int64) = 0.05 + 0.01/8*2^(j/5)
;

In [10]:
for j=1:af_len
    @printf("af = %f \n", af_list[j])
    idx1 = set_cstart_idx(j, data)
    temp = findfirst(x -> x > ctop_list[idx1] + cspan(j), ctop_list)
    idx2 = temp == nothing ? c_len : temp
    
    @time pmap((args)->inner_loop(args...), [(i, j, data) for i in idx1:idx2])
end

af = 20.000000 
      From worker 4:	ctop = 0.3925 : cT = 0.53558, minphi = 0.05155, zR = 0.20597, mh = 0.12506, valid 
      From worker 3:	ctop = 0.3950 : cT = 0.54037, minphi = 0.05570, zR = 0.22066, mh = 0.12510, valid 
      From worker 2:	ctop = 0.3900 : cT = 0.53078, minphi = 0.04753, zR = 0.19152, mh = 0.12510, valid 
      From worker 4:	ctop = 0.3975 : cT = 0.54520, minphi = 0.06062, zR = 0.23809, mh = 0.12509, valid 
      From worker 3:	ctop = 0.4000 : cT = 0.55002, minphi = 0.06575, zR = 0.25596, mh = 0.12508, valid 
      From worker 2:	ctop = 0.4025 : cT = 0.55487, minphi = 0.07145, zR = 0.27572, mh = 0.12506, valid 
      From worker 3:	ctop = 0.4075 : cT = 0.56470, minphi = 0.08478, zR = 0.32128, mh = 0.12509, valid 
      From worker 2:	ctop = 0.4100 : cT = 0.56970, minphi = 0.09249, zR = 0.34724, mh = 0.12508, valid 
      From worker 4:	ctop = 0.4050 : cT = 0.55975, minphi = 0.07774, zR = 0.29732, mh = 0.12510, valid 
      From worker 3:	ctop = 0.4125 : cT = 0.5747

      From worker 2:	ctop = 0.4300 : cT = 0.59265, minphi = 0.14586, zR = 0.52225, mh = 0.12507, valid 
      From worker 4:	ctop = 0.4325 : cT = 0.59863, minphi = 0.16004, zR = 0.56649, mh = 0.12509, valid 
      From worker 3:	ctop = 0.4350 : cT = 0.60486, minphi = 0.17517, zR = 0.61269, mh = 0.12511, valid 
      From worker 2:	ctop = 0.4375 : cT = 0.61157, minphi = 0.19288, zR = 0.66594, mh = 0.12512, valid 
      From worker 4:	ctop = 0.4400 : cT = 0.61890, minphi = 0.21320, zR = 0.72572, mh = 0.12510, valid 
      From worker 3:	ctop = 0.4425 : cT = 0.62695, minphi = 0.23632, zR = 0.79193, mh = 0.12511, valid 
      From worker 2:	ctop = 0.4450 : cT = 0.63672, minphi = 0.26637, zR = 0.87593, mh = 0.12506, valid 
      From worker 4:	ctop = 0.4475 : cT = 0.64844, minphi = 0.30226, zR = 0.97174, mh = 0.12509, valid 
      From worker 3:	ctop = 0.4500 : cT = 0.66992, minphi = 0.37001, zR = 1.14289, mh = 0.12507, valid 
      From worker 2:	ctop = 0.4525 : cT = 0.69678, minphi = 0.44

      From worker 3:	ctop = 0.4600 : cT = 0.69287, minphi = 0.44130, zR = 1.30540, mh = 0.12601, invalid 
      From worker 2:	ctop = 0.4625 : cT = 0.69632, minphi = 0.44130, zR = 1.29407, mh = 0.12716, invalid 
      From worker 4:	ctop = 0.4650 : cT = 0.69971, minphi = 0.44130, zR = 1.28281, mh = 0.12833, invalid 
 31.428194 seconds (7.30 k allocations: 423.328 KiB)
af = 16.500000 
      From worker 2:	ctop = 0.4150 : cT = 0.53296, minphi = 0.06056, zR = 0.24044, mh = 0.12508, valid 
      From worker 3:	ctop = 0.4125 : cT = 0.52805, minphi = 0.05591, zR = 0.22389, mh = 0.12510, valid 
      From worker 4:	ctop = 0.4175 : cT = 0.53787, minphi = 0.06550, zR = 0.25781, mh = 0.12511, valid 
      From worker 2:	ctop = 0.4200 : cT = 0.54285, minphi = 0.07150, zR = 0.27897, mh = 0.12509, valid 
      From worker 4:	ctop = 0.4250 : cT = 0.55286, minphi = 0.08447, zR = 0.32368, mh = 0.12507, valid 
      From worker 3:	ctop = 0.4225 : cT = 0.54782, minphi = 0.07762, zR = 0.30013, mh = 0.125

      From worker 3:	ctop = 0.4325 : cT = 0.54492, minphi = 0.08152, zR = 0.31524, mh = 0.12509, valid 
      From worker 4:	ctop = 0.4275 : cT = 0.53485, minphi = 0.06912, zR = 0.27207, mh = 0.12511, valid 
      From worker 2:	ctop = 0.4300 : cT = 0.53986, minphi = 0.07488, zR = 0.29216, mh = 0.12511, valid 
      From worker 3:	ctop = 0.4350 : cT = 0.55005, minphi = 0.08892, zR = 0.34075, mh = 0.12508, valid 
      From worker 4:	ctop = 0.4375 : cT = 0.55521, minphi = 0.09660, zR = 0.36675, mh = 0.12510, valid 
      From worker 2:	ctop = 0.4400 : cT = 0.56049, minphi = 0.10546, zR = 0.39658, mh = 0.12509, valid 
      From worker 3:	ctop = 0.4425 : cT = 0.56586, minphi = 0.11503, zR = 0.42835, mh = 0.12510, valid 
      From worker 4:	ctop = 0.4450 : cT = 0.57135, minphi = 0.12544, zR = 0.46245, mh = 0.12511, valid 
      From worker 2:	ctop = 0.4475 : cT = 0.57706, minphi = 0.13727, zR = 0.50076, mh = 0.12507, valid 
      From worker 3:	ctop = 0.4500 : cT = 0.58301, minphi = 0.15

      From worker 4:	ctop = 0.4450 : cT = 0.54507, minphi = 0.09082, zR = 0.34920, mh = 0.12508, valid 
      From worker 2:	ctop = 0.4525 : cT = 0.56116, minphi = 0.11788, zR = 0.44043, mh = 0.12507, valid 
      From worker 3:	ctop = 0.4550 : cT = 0.56671, minphi = 0.12817, zR = 0.47410, mh = 0.12506, valid 
      From worker 4:	ctop = 0.4575 : cT = 0.57251, minphi = 0.14031, zR = 0.51352, mh = 0.12508, valid 
      From worker 3:	ctop = 0.4625 : cT = 0.58472, minphi = 0.16758, zR = 0.59968, mh = 0.12512, valid 
      From worker 2:	ctop = 0.4600 : cT = 0.57849, minphi = 0.15333, zR = 0.55504, mh = 0.12509, valid 
      From worker 4:	ctop = 0.4650 : cT = 0.59131, minphi = 0.18354, zR = 0.64881, mh = 0.12511, valid 
      From worker 2:	ctop = 0.4700 : cT = 0.60645, minphi = 0.22451, zR = 0.77151, mh = 0.12506, valid 
      From worker 3:	ctop = 0.4675 : cT = 0.59839, minphi = 0.20177, zR = 0.70388, mh = 0.12512, valid 
      From worker 4:	ctop = 0.4725 : cT = 0.61499, minphi = 0.24

...

      From worker 4:	ctop = 0.5650 : cT = 0.43488, minphi = 0.07156, zR = 0.30449, mh = 0.12506, valid 
      From worker 2:	ctop = 0.5700 : cT = 0.44641, minphi = 0.08243, zR = 0.34523, mh = 0.12510, valid 
      From worker 3:	ctop = 0.5675 : cT = 0.44061, minphi = 0.07654, zR = 0.32313, mh = 0.12511, valid 
      From worker 4:	ctop = 0.5725 : cT = 0.45224, minphi = 0.08886, zR = 0.36915, mh = 0.12509, valid 
      From worker 3:	ctop = 0.5775 : cT = 0.46399, minphi = 0.10309, zR = 0.42117, mh = 0.12511, valid 
      From worker 2:	ctop = 0.5750 : cT = 0.45807, minphi = 0.09552, zR = 0.39357, mh = 0.12512, valid 
      From worker 4:	ctop = 0.5800 : cT = 0.46997, minphi = 0.11118, zR = 0.45031, mh = 0.12510, valid 
      From worker 3:	ctop = 0.5825 : cT = 0.47607, minphi = 0.12024, zR = 0.48271, mh = 0.12509, valid 
      From worker 2:	ctop = 0.5850 : cT = 0.48230, minphi = 0.13005, zR = 0.51733, mh = 0.12506, valid 
      From worker 4:	ctop = 0.5875 : cT = 0.48853, minphi = 0.13

      From worker 3:	ctop = 0.6025 : cT = 0.48413, minphi = 0.14806, zR = 0.58365, mh = 0.12511, valid 
      From worker 4:	ctop = 0.6050 : cT = 0.49097, minphi = 0.16087, zR = 0.62775, mh = 0.12508, valid 
      From worker 4:	ctop = 0.6125 : cT = 0.51270, minphi = 0.20345, zR = 0.76814, mh = 0.12511, valid 
      From worker 3:	ctop = 0.6100 : cT = 0.50513, minphi = 0.18800, zR = 0.71811, mh = 0.12509, valid 
      From worker 2:	ctop = 0.6075 : cT = 0.49780, minphi = 0.17322, zR = 0.66904, mh = 0.12509, valid 
      From worker 4:	ctop = 0.6150 : cT = 0.52100, minphi = 0.22177, zR = 0.82660, mh = 0.12509, valid 
      From worker 2:	ctop = 0.6200 : cT = 0.54004, minphi = 0.26613, zR = 0.96271, mh = 0.12508, valid 
      From worker 3:	ctop = 0.6175 : cT = 0.52979, minphi = 0.24133, zR = 0.88723, mh = 0.12509, valid 
      From worker 2:	ctop = 0.6250 : cT = 0.56641, minphi = 0.33079, zR = 1.14772, mh = 0.12511, valid 
      From worker 4:	ctop = 0.6225 : cT = 0.55176, minphi = 0.29

      From worker 4:	ctop = 0.6525 : cT = 0.61572, minphi = 0.44078, zR = 1.40337, mh = 0.12869, invalid 
      From worker 3:	ctop = 0.6550 : cT = 0.61938, minphi = 0.44130, zR = 1.39383, mh = 0.12972, invalid 
      From worker 2:	ctop = 0.6575 : cT = 0.62256, minphi = 0.44051, zR = 1.38137, mh = 0.13073, invalid 
      From worker 4:	ctop = 0.6600 : cT = 0.62622, minphi = 0.44103, zR = 1.37192, mh = 0.13178, invalid 
      From worker 3:	ctop = 0.6625 : cT = 0.62939, minphi = 0.44051, zR = 1.36018, mh = 0.13284, invalid 
      From worker 2:	ctop = 0.6650 : cT = 0.63306, minphi = 0.44103, zR = 1.35081, mh = 0.13391, invalid 
      From worker 4:	ctop = 0.6675 : cT = 0.63623, minphi = 0.44051, zR = 1.33919, mh = 0.13498, invalid 
      From worker 2:	ctop = 0.6725 : cT = 0.64307, minphi = 0.44051, zR = 1.31840, mh = 0.13717, invalid 
      From worker 3:	ctop = 0.6700 : cT = 0.63989, minphi = 0.44103, zR = 1.32990, mh = 0.13607, invalid 
      From worker 4:	ctop = 0.6750 : cT = 0.64

      From worker 3:	ctop = 0.6025 : cT = 0.29985, minphi = 0.03480, zR = 0.16637, mh = 0.12508, valid 
      From worker 4:	ctop = 0.6000 : cT = 0.29280, minphi = 0.03280, zR = 0.15781, mh = 0.12512, valid 
      From worker 2:	ctop = 0.6050 : cT = 0.30676, minphi = 0.03688, zR = 0.17525, mh = 0.12506, valid 
      From worker 3:	ctop = 0.6075 : cT = 0.31354, minphi = 0.03895, zR = 0.18387, mh = 0.12508, valid 
      From worker 4:	ctop = 0.6100 : cT = 0.32021, minphi = 0.04123, zR = 0.19339, mh = 0.12510, valid 
      From worker 2:	ctop = 0.6125 : cT = 0.32678, minphi = 0.04378, zR = 0.20407, mh = 0.12510, valid 
      From worker 3:	ctop = 0.6150 : cT = 0.33325, minphi = 0.04635, zR = 0.21463, mh = 0.12509, valid 
      From worker 4:	ctop = 0.6175 : cT = 0.33966, minphi = 0.04937, zR = 0.22711, mh = 0.12508, valid 
      From worker 3:	ctop = 0.6225 : cT = 0.35223, minphi = 0.05576, zR = 0.25316, mh = 0.12509, valid 
      From worker 2:	ctop = 0.6200 : cT = 0.34598, minphi = 0.05

      From worker 3:	ctop = 0.7025 : cT = 0.45251, minphi = 0.20224, zR = 0.80131, mh = 0.12509, valid 
      From worker 2:	ctop = 0.7050 : cT = 0.46045, minphi = 0.21711, zR = 0.85113, mh = 0.12508, valid 
      From worker 4:	ctop = 0.7075 : cT = 0.46875, minphi = 0.23302, zR = 0.90325, mh = 0.12509, valid 
      From worker 3:	ctop = 0.7100 : cT = 0.47754, minphi = 0.25023, zR = 0.95830, mh = 0.12511, valid 
 76.847320 seconds (12.99 k allocations: 704.594 KiB)


In [11]:
using DelimitedFiles
writedlm("data_uncolored.csv", data)

In [12]:
using JLD2
using FileIO

address = ""

save(address*"data_uncolored.jld2", 
    "ctop", data[ctop,:,:],
    "cT", data[cT,:,:],
    "aL", data[aL,:,:],
    "aB", data[aB,:,:],
    "af", data[af,:,:],
    "minphi", data[minphi,:,:],
    "higgsmass", data[higgsmass,:,:],
    "zR", data[zR,:,:],
    "af_list", af_list,
    "ctop_list", ctop_list)